In [1]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine, text
import os
import json

# !pip install "cloud-sql-python-connector[pg8000]"
from google.cloud.sql.connector import Connector

ModuleNotFoundError: No module named 'pandas'

In [12]:
def loadJSON(path):
    """
    Returns data from JSON file @ path
    """
    return json.load(open(path))

In [13]:
# Create engine
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/googleCloudKey.json'

def initPostgreSQL():
    """
    Connects and initializes PostgreSQL
    """
    # Import database creditations
    creds = loadJSON("/content/postgreSQLKey.json")

    def getConnection():
        connector = Connector()
        connection = connector.connect(
            creds["connectionName"],
            "pg8000",
            user=creds["user"],
            password=creds["password"],
            db=creds["dbname"]
        )
        return connection
    
    try: # Try to connecto to database
        engine = create_engine(
            "postgresql+pg8000://",
            creator=getConnection,
        )
        return engine
    except Exception as e: # Connection failed
        return None

In [15]:
# Get engine
engine = initPostgreSQL()

In [ ]:
# Load data
query = text("""
SELECT 
    leagues.name as league_name,
    COUNT(matches.id) as total_matches,
    SUM(CASE WHEN home_goals > away_goals THEN 1 ELSE 0 END) as home_wins,
    SUM(CASE WHEN home_goals = away_goals THEN 1 ELSE 0 END) as draws,
    SUM(CASE WHEN home_goals < away_goals THEN 1 ELSE 0 END) as away_wins
FROM matches
JOIN leagues ON matches.league_id = leagues.id
GROUP BY leagues.name;
""")
with engine.connect() as connection:
  df = pd.read_sql_query(query, connection)

print(df.head(5))

In [ ]:
# Plot data
win_data = pd.read_sql(query, con=engine)

win_data['home_win_percentage'] = (win_data['home_wins'] / win_data['total_matches']) * 100
win_data['draw_percentage'] = (win_data['draws'] / win_data['total_matches']) * 100
win_data['away_win_percentage'] = (win_data['away_wins'] / win_data['total_matches']) * 100

n_leagues = len(win_data)
index = np.arange(n_leagues)
bar_width = 0.3

fig, ax = plt.subplots(figsize=(12, 6))

rects1 = ax.bar(index, win_data['home_win_percentage'], bar_width, label='Home Win Percentage')
rects2 = ax.bar(index + bar_width, win_data['draw_percentage'], bar_width, label='Draw Percentage')
rects3 = ax.bar(index + 2 * bar_width, win_data['away_win_percentage'], bar_width, label='Away Win Percentage')

ax.set_xlabel('League')
ax.set_ylabel('Percentage')
ax.set_title('Outcome Percentages by League')
ax.set_xticks(index + bar_width)
ax.set_xticklabels(win_data['league_name'])
ax.legend()

fig.tight_layout()
plt.show()